In [303]:
from IPython import display
import math
from pprint import pprint
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

In [304]:
import praw
from psaw import PushshiftAPI

api = PushshiftAPI()

import datetime as dt

start_epoch=int(dt.datetime(2021,4,1).timestamp())
end_epoch1 = int(dt.datetime(2021, 7,30).timestamp())

list1=list(api.search_submissions(after=start_epoch,before =end_epoch1, 
                            subreddit='worldnews',
                            filter=['title']))

/opt/anaconda3/lib/python3.7/site-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/opt/anaconda3/lib/python3.7/site-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


In [305]:
new1=[None]*len(list1)

In [306]:
for i in range(0,len(list1)):
    new1[i]=list1[i][3]['title']

s=new1
new=s

In [307]:
for i in range(0,len(s)):
    txt=s[i]
    s[i]=s[i].replace('%','percent')

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize


for i in range(0,len(s)):
    text = s[i]
    text_tokens = word_tokenize(text)

    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    new[i]=' '.join(tokens_without_sw)

import nltk
nltk.download('averaged_perceptron_tagger')
for i in range(0,len(s)):   
    sentence = new[i]
    tagged_sentence = nltk.tag.pos_tag(sentence.split())
    edited_sentence = [word for word,tag in tagged_sentence if tag != 'JJ' and tag != 'JJR' and tag != 'JJS' and tag != 'RB' and tag != 'RBR' and tag != 'RBS']
    new1[i]=' '.join(edited_sentence)
    
   

In [313]:
new1=list(filter(lambda a: a != '', new1))

In [314]:
 

import nltk
from pycorenlp import *
import collections

l = [None] * (len(new1))

for i in range(0,(len(new1))):
    nlp=StanfordCoreNLP('http://localhost:9000/')
    x=new1[i]
    output = nlp.annotate(x, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie","triple.strict":"true",
                                     "outputFormat": "json",
                                        "openie.max_entailments_per_clause":"1"})
    
    result = [output["sentences"][0]["openie"] for item in output]
    if len(result[0])>0:
        z=len(result[0])
        relationSent=result[0][z-1]['relation'],result[0][z-1]['subject'],result[0][z-1]['object']
        l[i]=list(relationSent)
    else:
        l[i]=[None,None,None]
       

In [315]:
l.reverse()  
df = pd.DataFrame(l, columns =['P', 'S', 'O'], dtype = float)

In [316]:
df1 = df.replace(to_replace='None', value=np.nan).dropna()
df1.shape[0]/df.shape[0]



0.4798793859649123

In [317]:
triples=df1.values

In [318]:
import math
train=df1[:math.floor(0.75*len(df1))]
valid=df1[math.floor(0.75*len(df1)):math.floor(0.90*len(df1))]
test=df1[math.floor(0.9*len(df1)):]

In [319]:
train_triples=train.values
valid_triples=valid.values
test_triples=test.values

In [320]:
all_data=df1.values

In [322]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

urls=[]

for z in range(0,len(train_triples)):
    try:
        ## initial consts
        BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
        TEXT = train_triples[z][2]
        CONFIDENCE = '0.9'
        SUPPORT = '10'
        REQUEST = BASE_URL.format(
            text=urllib.parse.quote_plus(TEXT), 
            confidence=CONFIDENCE, 
            support=SUPPORT
        )
        HEADERS = {'Accept': 'application/json'}
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        all_urls = []

        r = requests.get(url=REQUEST, headers=HEADERS)
        response = r.json()
        resources = response['Resources']
        for res in resources:
            all_urls.append(res['@URI'])
        
        urls = all_urls + urls

    except:
        
        pass # doing nothing on exception
    

In [323]:
urls = list(dict.fromkeys(urls))
test_urls=urls

In [324]:

    columns = ['S','P', 'O']
    data = []
    whole_df= pd.DataFrame(data,columns=columns)
    for i in range(0,len(test_urls)):
                
                line = 'DESCRIBE '
                output_line=line +'<' + test_urls[i] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
                
                

                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for i in range(1,(len(results['results']['bindings']))):

                    s_l[i]=results['results']['bindings'][i]['s']['value']
                    p_l[i]=results['results']['bindings'][i]['p']['value']
                    o_l[i]=results['results']['bindings'][i]['o']['value']
                    
                    

                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                               columns =['S', 'P','O'])
                extra_df=extra_df.iloc[1:]
                
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']
                
                
                
                
                
                
                frames=[whole_df,extra_df]

                whole_df=pd.concat(frames)

Exception ignored in: <function SeekableUnicodeStreamReader.__del__ at 0x7fe27aad7320>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/site-packages/nltk/data.py", line 1280, in __del__
    if not self.closed:
  File "/opt/anaconda3/lib/python3.7/site-packages/nltk/data.py", line 1294, in closed
    return self.stream.closed
AttributeError: 'SeekableUnicodeStreamReader' object has no attribute 'stream'


In [325]:
test=whole_df.groupby('P').count()
test.sort_values(by=['O'])

,S,O
P,,
http://dbpedia.org/ontology/Automobile/wheelbase,1,1
http://dbpedia.org/property/firstFlightDate,1,1
http://dbpedia.org/property/firstHatTrickPlayer,1,1
http://dbpedia.org/property/firstHolder,1,1
http://dbpedia.org/property/firstLaunch,1,1
...,...,...
http://dbpedia.org/property/location,172780,172780
http://www.w3.org/2002/07/owl#sameAs,262779,262779
http://dbpedia.org/property/birthPlace,281440,281440


In [25]:
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import urllib.parse

valid_urls=[]

for z in range(0,len(valid_triples)):
    try:
        ## initial consts
        BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
        TEXT = valid_triples[z][2]
        CONFIDENCE = '0.9'
        SUPPORT = '10'
        REQUEST = BASE_URL.format(
            text=urllib.parse.quote_plus(TEXT), 
            confidence=CONFIDENCE, 
            support=SUPPORT
        )
        HEADERS = {'Accept': 'application/json'}
        sparql = SPARQLWrapper("http://dbpedia.org/sparql")
        all_urls = []

        r = requests.get(url=REQUEST, headers=HEADERS)
        response = r.json()
        resources = response['Resources']
        for res in resources:
            all_urls.append(res['@URI'])
        
        valid_urls = all_urls + valid_urls

    except:
        
        pass # doing nothing on exception

In [27]:
    valid_urls = list(dict.fromkeys(valid_urls))
    columns = ['S','P', 'O']
    data = []
    valid_df= pd.DataFrame(data,columns=columns)
    for i in range(0,len(valid_urls)):
                
                line = 'DESCRIBE '
                output_line=line +'<' + valid_urls[i] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()
                
                

                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for i in range(1,(len(results['results']['bindings']))):

                    s_l[i]=results['results']['bindings'][i]['s']['value']
                    p_l[i]=results['results']['bindings'][i]['p']['value']
                    o_l[i]=results['results']['bindings'][i]['o']['value']
                    
                    

                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                               columns =['S', 'P','O'])
                extra_df=extra_df.iloc[1:]
                
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']
                
                
                
                
                extra_df['O'] = extra_df['O'].astype(str)
                extra_df=extra_df[extra_df['S'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['P'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['O'].str.contains("http://dbpedia.org/",na=False)]
                
                frames=[valid_df,extra_df]

                valid_df=pd.concat(frames)
                

In [328]:
final_df=pd.concat([train, whole_df], sort=False)

In [28]:
valid_final_df=pd.concat([valid, valid_df], sort=False)

In [329]:
final_df["S"] = final_df["S"].astype(str)
final_df["P"] = final_df["P"].astype(str)
final_df["O"] = final_df["O"].astype(str)

In [30]:
valid_final_df["S"] = valid_final_df["S"].astype(str)
valid_final_df["P"] = valid_final_df["P"].astype(str)
valid_final_df["O"] = valid_final_df["O"].astype(str)

In [36]:

#test2=valid_final_df.groupby('P').count()
#test2=test2.sort_values(by=['O'])
#first_v=test2.iloc[:, 0]
#relations_valid=first_v.index.values.tolist() , 
#list(set(relations_valid) - set(relations_train))

In [330]:
final_df = final_df[['S','P','O']]

In [37]:
valid_final_df = valid_final_df[['S','P','O']]

In [331]:
remove_rel=final_df.groupby('P').count()
remove_rel=remove_rel.sort_values(by=['O'])

In [332]:
single_rel=remove_rel.loc[remove_rel['S'] < 10]
single_rel=single_rel.index.values.tolist()

final_df=final_df[~final_df['P'].isin(single_rel)]
final_df

,S,P,O
15,Analogue,’s,Pocket
16,E!,has,Erin Lim Rhodes
17,Amy Winehouse,has,Mom
18,Biden,’s,Big Bet
19,PingPong,is,video chat app for product teams working acros...
...,...,...,...
854,http://dbpedia.org/resource/Alexa_Internet,http://dbpedia.org/property/currentStatus,Online
861,http://dbpedia.org/resource/Alexa_Internet,http://www.w3.org/2002/07/owl#sameAs,http://lt.dbpedia.org/resource/Alexa_Internet
866,http://dbpedia.org/resource/Alexa_Internet,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://dbpedia.org/class/yago/Abstraction10000...
868,http://dbpedia.org/resource/Alexa_Internet,http://www.w3.org/2002/07/owl#sameAs,http://ja.dbpedia.org/resource/アレクサ・インターネット


In [333]:
#remove relations in validation set which do not exist in training set

test1=final_df.groupby('P').count()
test1=test1.sort_values(by=['O'])
first_c=test1.iloc[:, 0]
relations_train=first_c.index.values.tolist()

#valid_final_df=valid_final_df[valid_final_df['P'].isin(relations_train)]

In [334]:
test2=final_df.groupby('S').count()
test2=test2.sort_values(by=['O'])
first_c_s=test2.iloc[:, 0]
subject_train=first_c_s.index.values.tolist()
#valid_final_df=valid_final_df[valid_final_df['S'].isin(subject_train)]

In [335]:
test3=final_df.groupby('O').count()
test3=test3.sort_values(by=['S'])
first_c_o=test3.iloc[:, 0]
object_train=first_c_o.index.values.tolist()

In [337]:
import ampligraph
ampligraph.latent_features.set_entity_threshold(2276568)

In [338]:
import numpy as np

from ampligraph.latent_features import TransE

train_all_triple=final_df.values
#valid_all_triple=valid_final_df.values

model = TransE(batches_count=64, seed=0, epochs=30, k=100, eta=40,optimizer='adam', optimizer_params={'lr':0.0001},loss='pairwise', verbose=True)

model.fit(train_all_triple) #, early_stopping=True, early_stopping_params={'x_valid':valid_all_triple,'burn_in':100}

Average TransE Loss:   0.035257: 100%|██████████| 30/30 [3:01:21<00:00, 362.73s/epoch]  


In [80]:
example_name = 'helloworld.pkl'
ampligraph.utils.save_model(model, model_name_path = example_name)

In [82]:
final_df.to_pickle("final_df.pkl")
valid_final_df.to_pickle("valid_final_df.pkl")

<font size="7">Daily Headline Data</font>

In [ ]:
start_date = dt.date(2021, 7, 1)
end_date = dt.date(2021, 8, 8)
x=(end_date - start_date)
avg_scores=[None]*(x.days) 
dayy=0

while start_date < end_date:
    
    next_day=start_date+delta
    reddit = praw.Reddit(
    client_id='IKnXnWxiUuXhKA',
                         client_secret='wvvyNqFcSHiTLOgfTGcI_FIsaYIyew',
                         user_agent='tm_quant'
    )
    api = PushshiftAPI(reddit)

    start_epoch = int(pd.Timestamp(start_date).timestamp())
    end_epoch = int(pd.Timestamp(next_day).timestamp())

    gen = api.search_submissions(
      subreddit = 'worldnews',
      filter = ['id'],
      after = start_epoch,
      before = end_epoch
    )

    results = list(gen)


    titles=results
    for i in range(0,len(titles)):
        titles[i]=titles[i].title

    reddit = praw.Reddit(
      client_id='IKnXnWxiUuXhKA',
                         client_secret='wvvyNqFcSHiTLOgfTGcI_FIsaYIyew',
                         user_agent='tm_quant'
    )
    api = PushshiftAPI(reddit)

    start_epoch = int(pd.Timestamp(start_date).timestamp())
    end_epoch = int(pd.Timestamp(next_day).timestamp())

    gen = api.search_submissions(
      subreddit = 'worldnews',
      filter = ['id'],
      after = start_epoch,
      before = end_epoch
    )

    results = list(gen)

    scores=results
    for r in range(0,len(titles)):
        scores[r]=scores[r].score


    #x=len(scores)
    #y=len(titles)
    #scores=(scores[(x-len(titles)):x])

    todays_news_dict = {'Title':titles,'Score':scores}
    todays_news = pd.DataFrame(todays_news_dict)

    todays_news=todays_news.sort_values(by=['Score'], ascending=False)

    todays_news=todays_news.Title.values.tolist()

    top_news=todays_news[0:50]

    headlines=top_news
    s = top_news
    new=s
    new1=s

    for y in range(0,len(s)):
        txt=s[y]
        s[y]=s[y].replace('%','percent')




    from nltk.corpus import stopwords
    nltk.download('stopwords')
    from nltk.tokenize import word_tokenize


    for u in range(0,len(s)):
        text = s[u]
        text_tokens = word_tokenize(text)

        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
        new[u]=' '.join(tokens_without_sw)

    import nltk
    nltk.download('averaged_perceptron_tagger')
    for c in range(0,len(s)):   
        sentence = new[c]
        tagged_sentence = nltk.tag.pos_tag(sentence.split())
        edited_sentence = [word for word,tag in tagged_sentence if tag != 'JJ' and tag != 'JJR' and tag != 'JJS' and tag != 'RB' and tag != 'RBR' and tag != 'RBS']
        new1[c]=' '.join(edited_sentence)

    import nltk
    from pycorenlp import *
    import collections

    l = [None] * (len(headlines))

    for g in range(0,len(headlines)):
        nlp=StanfordCoreNLP('http://localhost:9000/')
        x=new1[g]
        output = nlp.annotate(x, properties={"annotators":"tokenize,ssplit,pos,depparse,natlog,openie","triple.strict":"true",
                                         "outputFormat": "json",
                                            "openie.max_entailments_per_clause":"1"})

        result = [output["sentences"][0]["openie"] for item in output]
        if len(result[0])>0:
            z=len(result[0])
            relationSent=result[0][z-1]['relation'],result[0][z-1]['subject'],result[0][z-1]['object']
            l[g]=list(relationSent)
        else:
            l[g]=[None,None,None]

    day_df = pd.DataFrame(l, columns =['P', 'S', 'O'], dtype = float)
    day_df = day_df.replace(to_replace='None', value=np.nan).dropna()



    day_triples=day_df.values

    import json
    from SPARQLWrapper import SPARQLWrapper, JSON
    import requests
    import urllib.parse

    day_urls=[]

    for p in range(0,len(day_triples)):
        try:
            ## initial consts
            BASE_URL = 'http://api.dbpedia-spotlight.org/en/annotate?text={text}&confidence={confidence}&support={support}'
            TEXT = day_triples[p][2]
            CONFIDENCE = '0.6'
            SUPPORT = '10'
            REQUEST = BASE_URL.format(
                text=urllib.parse.quote_plus(TEXT), 
                confidence=CONFIDENCE, 
                support=SUPPORT
            )
            HEADERS = {'Accept': 'application/json'}
            sparql = SPARQLWrapper("http://dbpedia.org/sparql")
            all_urls = []

            r = requests.get(url=REQUEST, headers=HEADERS)
            response = r.json()
            resources = response['Resources']
            for res in resources:
                all_urls.append(res['@URI'])

            day_urls = all_urls + day_urls

        except:

            pass # doing nothing on exception
        

    day_urls = list(dict.fromkeys(day_urls))
    



    columns = ['S','P', 'O']
    data = []
    whole_day_df= pd.DataFrame(data,columns=columns)
    for m in range(0,len(day_urls)):

                line = 'DESCRIBE '
                output_line=line +'<' + day_urls[m] + '>'



                sparql = SPARQLWrapper("http://dbpedia.org/sparql")

                q="""

                    {}

                """.format(output_line)

                sparql.setQuery(q)



                sparql.setReturnFormat(JSON)
                results = sparql.query().convert()



                s_l = [None] * (len(results['results']['bindings']))
                p_l = [None] * (len(results['results']['bindings']))
                o_l = [None] * (len(results['results']['bindings']))
                for t in range(1,(len(results['results']['bindings']))):

                    s_l[t]=results['results']['bindings'][t]['s']['value']
                    p_l[t]=results['results']['bindings'][t]['p']['value']
                    o_l[t]=results['results']['bindings'][t]['o']['value']



                extra_df = pd.DataFrame(list(zip(s_l, p_l,o_l)),
                                   columns =['S', 'P','O'])
                extra_df=extra_df.iloc[t:]

                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageWikiLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageExternalLink']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageLength']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/ontology/wikiPageRedirects']
                extra_df = extra_df[extra_df.P != 'http://dbpedia.org/property/wikiPageUsesTemplate']


                
                extra_df['O'] = extra_df['O'].astype(str)
                extra_df=extra_df[extra_df['S'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['P'].str.contains("http://dbpedia.org/",na=False)]
                extra_df=extra_df[extra_df['O'].str.contains("http://dbpedia.org/",na=False)]



                frames=[whole_day_df,extra_df]

                whole_day_df=pd.concat(frames)

    whole_day_df=whole_day_df[whole_day_df['P'].isin(relations_train)]
    whole_day_df=whole_day_df[whole_day_df['S'].isin(subject_train)]
    whole_day_df=whole_day_df[whole_day_df['O'].isin(object_train)]

    ##check=whole_day_df.groupby('O').count()
    #check=check.sort_values(by=['S'])
    #check_s=check.iloc[:, 0]
    #relations_day=check_s.index.values.tolist()
    #list(set(relations_day) - set(object_train))

    day_triples_final=whole_day_df.values

    
    if len(day_triples_final)==0:
        avg_scores[dayy]=0
    
    else:
        scores = model.predict(day_triples_final)
        import statistics
        avg_scores[dayy]=statistics.mean(scores)
    
        
    start_date += delta
    dayy=dayy+1

In [542]:
start_date = dt.date(2021, 7, 1)
end_date = dt.date(2021, 8, 8)


number_of_days = end_date-start_date
number_of_days=number_of_days.days


date_list = []
for day in range(number_of_days):
    a_date = (start_date + dt.timedelta(days = day)).isoformat()
    date_list.append(a_date)


In [544]:
daily_df = pd.DataFrame(
    {'Date': date_list,
     'score': avg_scores
    })

In [545]:
daily_df['Date']= pd.to_datetime(daily_df['Date'])

In [554]:
import yfinance as yf

msft = yf.Ticker("AAPL")

# get historical market data
hist = msft.history(period="27d")

date_price = hist[["Close"]]
date_price['Date'] = date_price.index
date_price=date_price[['Date','Close']]
date_price.index = range(0, len(date_price))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [555]:
price_lst=hist.Close.tolist()

up_down=price_lst[:-1]
for p in range(0,len(price_lst)-1):
    if price_lst[p+1]>price_lst[p]:
        up_down[p]=1
    else:
        up_down[p]=-1
        


In [556]:
final_daily_df=pd.merge(date_price, daily_df, on='Date')

final_daily_df['up_down'] = up_down

<font size="7">Prediction</font>


In [558]:
import xgboost as xgb

xgb_cl = xgb.XGBClassifier()

In [559]:
X = final_daily_df.drop(["Date",'Close','up_down'], axis=1)
y = final_daily_df.up_down

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=1121218
)

In [560]:
from sklearn.metrics import accuracy_score

# Init classifier
xgb_cl = xgb.XGBClassifier()

# Fit
xgb_cl.fit(X_train, y_train)

# Predict
preds = xgb_cl.predict(X_test)

# Score
accuracy_score(y_test, preds)

[22:01:59] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/opt/anaconda3/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


0.5714285714285714

In [614]:
from sklearn.model_selection import GridSearchCV 
from sklearn import svm
from sklearn.svm import SVC
import numpy as np
model = SVC() #[0.1, 1, 10, 100, 1000]
# defining parameter range 
param_grid = {'C': [0.1, 0.01, 0.001],  
              'gamma': [0.1,0.01,0.001],
              'degree':[2,3],
              'coef0':np.linspace(-200,200,20),
              'kernel': ['rbf', 'linear','poly']}  
  
grid = GridSearchCV(SVC(), param_grid, scoring='recall_macro') 
  
# fitting the model for grid search 
grid.fit(X_train, y_train) 

grid_predictions = grid.predict(X_test) 

/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [615]:
from sklearn import metrics

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.5714285714285714
